In [2]:
##training
import os,pickle
import pandas as pd
from utils.DataProcess import Start
from utils.ClooingTowerOptimize import CT_opt

opt = CT_opt()
opt.df = Start('HL_14')

opt.train()
with open(os.path.join('model','regr.pkl'), 'wb') as f:
    pickle.dump(opt, f)

冷卻水塔 MAE: 36.21
冰水主機 MAE: 41.75
冷卻水泵 MAE: 12.75


In [2]:
# eval plot
import os,pickle
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
from utils.DataProcess import Start
from utils.ClooingTowerOptimize import CT_opt
with open(os.path.join('model','regr.pkl'), 'rb') as p:
    opt = pickle.load(p)


vs = []
for c in tqdm(range(1000)):
    try:
        opt.predict(c)
        v,fig = opt.plot(21,29)
        vs.append(v)
    except:
        pass


fig = go.Figure()
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.condenser_supply_temp[-1000:][::-1],mode='lines',name='Manual'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=np.array(vs),mode='lines',name='AI'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.Wet_bulb_temp[-1000:][::-1],mode='lines',name='Wet bulb'))

100%|██████████| 1000/1000 [00:19<00:00, 50.90it/s]


In [22]:
import plotly.express as px
df = opt.df.copy()
df = df[df.CT_Total_KW>0]
px.scatter(data_frame=df,x='CT_eff',y='CT_Total_KW'
,color='Wet_bulb_temp'
)

In [3]:
a = opt.predict(1)
a,b = opt.plot(21,29)
b

In [4]:
import plotly.express as px
df = opt.df.copy()
df = df[df.Wet_bulb_temp>0]
px.scatter(
    data_frame=df,
    x='CT_09_VFD',
    y='Approach',
    color='Wet_bulb_temp'
)

In [5]:
import plotly.express as px
df14 = Start('HL_14').copy()
df14 = df14[df14.CT_09_VFD>47]
px.scatter(
    data_frame=df14,
    x='Wet_bulb_temp',
    y='Approach',
    color='CH14_Tune'
)

In [51]:
import sklearn
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split

def pp (df,Hz,col):
    df = df[(df[col]>Hz-1)&(df[col]<Hz+1)]
    X  = np.array(df.Enthalpy).reshape(-1, 1)
    y = np.array(df.Approach)
    ## 將數據集拆成訓練集與測試集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

    regr = sklearn.linear_model.TweedieRegressor(power=2, alpha=0, max_iter=10000)
    regr.fit(X_train,y=y_train)
    cv = sklearn.model_selection.KFold(n_splits=5,shuffle=True)
    y_cv = sklearn.model_selection.cross_val_predict(regr,X_test,y=y_test,cv=cv)
    mae = round(sklearn.metrics.mean_absolute_error(y_test,y_cv),2)
    print(mae)

    res = []
    RG = np.linspace(14,28)
    for i in RG:
        res.append(regr.predict(i.reshape(-1, 1)))
    res = np.array(res).flatten()

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(x = RG, y=res, mode='lines',name=f'{Hz}hz')
        )

    fig.add_trace(
    go.Scatter(x = df.Enthalpy, y=df.Approach, mode='markers',opacity=0.2,name=f'{Hz}hz',marker_color='red')
    )
    return go.Scatter(x = RG, y=res, mode='lines',name=f'{Hz}hz'),go.Scatter(x = df.Enthalpy, y=df.Approach, mode='markers',opacity=0.2,name=f'{Hz}hz')

In [52]:
df14 = Start('HL_14').copy()
df8 = Start('HL_8').copy()

In [53]:
fig = go.Figure()
fig.add_traces(pp(df14,48,'CT_09_VFD')[0])
fig.add_traces(pp(df14,48,'CT_09_VFD')[1])
fig.add_traces(pp(df14,45,'CT_09_VFD')[0])
fig.add_traces(pp(df14,45,'CT_09_VFD')[1])
fig.add_traces(pp(df14,30,'CT_09_VFD')[0])
fig.add_traces(pp(df14,30,'CT_09_VFD')[1])
fig.add_traces(pp(df14,20,'CT_09_VFD')[0])
fig.add_traces(pp(df14,20,'CT_09_VFD')[1])
# t2 = pp(df14,35,'CT_VFD_OUT')


0.44
0.44
0.52
0.52
0.42
0.41
0.74
0.79
